## TEST Code for running and evaluating Steadiness / Cohesiveness and other metrics

#### Import packages & set dataset path

In [1]:
from helper import *
import sys
import importlib
import numpy as np

## Import Metrics
sys.path.append("/home/hj/vis2021_snc/metrics4mdp/metrics4mdp")
from provider import MDPMetricProvider

## Import SNC
sys.path.append("/home/hj/vis2021_snc/steadiness_cohesiveness")
import snc as sc





PATH_TO_DATASET = "../../dataset-generator/dataset/"

#### Pilot Dataset Load

In [2]:
mnist_umap_path = make_path(PATH_TO_DATASET, "mnist", "umap", 1, [0.5, 10])
emb_data, raw_data, label_data = get_data(mnist_umap_path)

raw_data = np.array(raw_data)
emb_data = np.array(emb_data)
label_data = np.array(label_data)


print(emb_data[:4])
print(len(emb_data))

[[14.0834322   4.46289539]
 [ 8.24845791  0.88252968]
 [12.69062042  1.50016272]
 [-0.71390963  4.22037649]]
10000


#### Pilot Local Metrics Usage

In [3]:
# mlist = ["Sammon", "DTM", "DTM_KL1", "Trustworthiness", "Continuity", "MRRE_XZ", "MRRE_ZX"]
# metric_provider = MDPMetricProvider(raw_data, emb_data, mlist, 7) 
# result = metric_provider.run()
# print(result)

In [4]:
import time

## reload SNC for the testing
importlib.reload(sc)
SNC = sc.SNC

parameter = {
    "k": 50
}


start = time.time()
snc = SNC(raw_data, emb_data, iteration=500, cluster_parameter = parameter)
snc.fit()

print(time.time() - start)


10.305732250213623


In [ ]:
for _ in range(5):
    start = time.time()
    result = snc.steadiness()
    print(result)
    print(time.time() - start)
    
# result2 = snc.cohesiveness()
# print(len(result))
# print(label_data[result])

0.7838738737408177
116.24018836021423
0.7832085855233303
113.46308732032776
